In [88]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
import io
from io import StringIO
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import os
import matplotlib.pyplot as plt
#from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

~~ ML Test 1 #

In [58]:
ML_df = final_df.copy()

~~ ML Part ~

In [90]:
ML_df['Transaction Date'] = pd.to_datetime(ML_df['Transaction Date'], format='%d/%m/%Y', errors='coerce')
ML_df['Month'] = ML_df['Transaction Date'].dt.strftime('%Y-%m')
monthly_sales = ML_df.groupby(['Month', 'Code', 'Item_name'])['Out'].sum().reset_index()

In [ ]:
next_month_name = monthly_sales['Month'].max()
year, month = map(int, next_month_name.split('-'))
date = datetime(year, month, 1)
new_date = date + timedelta(days=31)
new_month_name = new_date.strftime('%Y-%m')

def forecast_sales(df):
    
    forecasts = []
    
    for item in df['Item_name'].unique():
        item_data = df[df['Item_name'] == item].copy()
        
        if len(item_data) < 2:
            continue
        
        item_data['Month_Index'] = range(1, len(item_data) + 1)
        
        X = item_data[['Month_Index']]
        y = item_data['Out']
    
        
        model = LinearRegression()
        model.fit(X, y)
            
        next_month_index = np.array([[item_data['Month_Index'].max() + 1]])
        next_month_sales_forecast = model.predict(next_month_index)
        
        forecasts.append((item, next_month_sales_forecast[0]))
    
    forecast_df = pd.DataFrame(forecasts, columns=['Item_name', new_month_name])
    
    return forecast_df 

In [ ]:
sales_forecasts = forecast_sales(monthly_sales)
sales_forecasts[new_month_name] = np.round(sales_forecasts[new_month_name], 0).astype(int)

In [128]:
monthly_sales_by_month = ML_df.groupby(['Month', 'Item_name','Category'])['Out'].sum().reset_index()
update_sales= monthly_sales_by_month.pivot(index=['Item_name','Category'], columns='Month', values='Out').fillna(0).reset_index()

In [126]:
joined_data = pd.merge(update_sales, sales_forecasts, on=["Item_name"], how="left").fillna(0)
joined_data['Category'] = update_sales['Category']

In [127]:
df= pd.DataFrame(joined_data)
df

,Item_name,Category,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09
0,Certis反光衣(軍) 2XL,反光衣 Reflective Vest,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,2.0
1,Certis反光衣(軍) 3XL,反光衣 Reflective Vest,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,Certis反光衣(軍) L,反光衣 Reflective Vest,5.0,4.0,2.0,4.0,5.0,5.0,1.0,1.0,3.0,10.0,4.0,3.0,7.0,5.0
3,Certis反光衣(軍) M,反光衣 Reflective Vest,5.0,7.0,2.0,1.0,6.0,3.0,2.0,2.0,3.0,7.0,4.0,1.0,3.0,3.0
4,Certis反光衣(軍) S,反光衣 Reflective Vest,1.0,3.0,2.0,4.0,3.0,2.0,2.0,2.0,1.0,7.0,0.0,2.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,黑色Certis Polo 恤衫 (軍) L,軍裝上衣 Military Top,0.0,0.0,0.0,6.0,0.0,0.0,4.0,2.0,0.0,0.0,5.0,0.0,1.0,1.0
553,黑色Certis Polo 恤衫 (軍) M,軍裝上衣 Military Top,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0
554,黑色Certis Polo 恤衫 (軍) S,軍裝上衣 Military Top,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0
555,黑色Certis Polo 恤衫 (軍) XL,軍裝上衣 Military Top,0.0,0.0,3.0,5.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,1.0


In [74]:
df.to_csv('forecasted(2).csv',encoding='utf-8', errors='strict',index=False)

In [75]:
tableau_df = pd.melt(df, id_vars=['Item_name', 'Category'], var_name='Month', value_name='Sales')

In [80]:
tableau_df.to_csv('all_month.csv', encoding = 'utf-8', errors ='strict', index=False)